## <b><center>CUSTOMER TRANSACTIONS</b></center>

##### <b>IMPORT LIBRARY PACKAGES</b>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.preprocessing import LabelEncoder 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

##### <b>LOAD & DISPLAY DATASET</b>

In [ ]:
customer_transactions = pd.read_csv("sample_dataset.csv")
customer_transactions

##### <b>DATA UNDERSTANDING</b>

The data consists of customer transaction records with the following columns:

* customer_id: Unique identifier for each customer.

* Name: First name of the customer.

* Surname: Last name of the customer.

* Gender: the sexual orientation of the customer.

* Birthdate: the day, month and year the customer was born.

* Transaction Amount: Amount of transaction made.
* Date: Date of the transaction.
* Merchant Name: the name of the merchant making the transactions.
* Category: Product category.

##### <b>DATA EXPLORATION</b>

CHECK THE SHAPE

In [ ]:
print('The number of rows and columns in this dataset is', customer_transactions.shape, 'respectively')

CHECK THE SIZE

In [ ]:
print('The size of this dataset is', customer_transactions.size) 

CHECK THE COLUMNS

In [ ]:
customer_transactions.columns

CHECK INFO

In [ ]:
customer_transactions.info()

In [ ]:
customer_transactions.describe(include='all').T 

##### <b>DATA PREPARATION</b>

CHECK FOR MISSING VALUES

In [ ]:
customer_transactions.isna()

In [ ]:
customer_transactions.isna().sum()

DISPLAY HEATMAP FOR MISSING VALUES

In [ ]:
sns.heatmap(customer_transactions.isna())

CHECK UNIQUE VALUES IN THE GENDER COLUMN

In [ ]:
customer_transactions['Gender'].unique()

CHECK THE VALUE COUNT OF EACH GENDER

In [ ]:
customer_transactions['Gender'].value_counts()

REPLACE THE INITIALS OF THE GENDER TO FULL NAME

In [ ]:
customer_transactions['Gender'] = customer_transactions['Gender'].replace('F', 'Female')
customer_transactions['Gender'] = customer_transactions['Gender'].replace('M', 'Male')

FILL NAN VALUES WITH UNKNOWN

In [ ]:
#Replace nan values in the Gender column with "Unknown"
customer_transactions['Gender'].fillna('Unknown', inplace=True)

CHECK VALUE COUNTS AFTER THE CORRECTION AND REPLACEMENT

In [ ]:
customer_transactions['Gender'].value_counts()

In [ ]:
customer_transactions.isnull().sum()

CREATE A NEW COLUMN FOR AGE

In [ ]:
#To further aid our analysis, I will be creating a new column that will generate the age of the customer as at the time of this analysis.

customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

customer_transactions['Age'] = customer_transactions['Birthdate'].apply(lambda x: (datetime.now() - x).days // 365)

In [ ]:
customer_transactions

In [ ]:
# Create histogram with KDE
fig = px.histogram(customer_transactions, x='Age', nbins=30, marginal='violin', title='Age Distribution')

# Show the plot
fig.show()

In [ ]:
# Create a dataframe for the bar chart
age_distribution = customer_transactions['Age'].value_counts(bins=30).sort_index().reset_index()
age_distribution.columns = ['Age Group', 'Count']

# Create bar chart
fig = px.bar(age_distribution, x='Age Group', y='Count', title='Age Distribution', labels={'Age Group': 'Age Group', 'Count': 'Count'})

# Show the plot
fig.show()

DATA CLEANING

In [ ]:
customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])
customer_transactions['Gender'] = customer_transactions['Gender'].astype('category')

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(customer_transactions['Transaction Amount'], bins=30, kde=True)
plt.title('Transaction Amount Distribution')

In [ ]:
plt.subplot(1, 2, 2)
sns.histplot(customer_transactions['Age'], bins=30, kde=True)
plt.title('Age Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x='Gender', data=customer_transactions)
plt.title('Gender Distribution')


In [ ]:
plt.subplot(1, 2, 2)
sns.countplot(y='Category', data=customer_transactions)
plt.title('Transaction Categories')
plt.show()

In [ ]:
customer_transactions['YearMonth'] = customer_transactions['Date'].dt.to_period('M')

plt.figure(figsize=(12, 5))
sns.lineplot(data=customer_transactions.groupby('YearMonth')['Transaction Amount'].sum().reset_index(), x='YearMonth', y='Transaction Amount')
plt.title('Monthly Transaction Amount Over Time')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(x='Gender', y='Transaction Amount', data=customer_transactions)
plt.title('Transaction Amount by Gender')
plt.show()

customer_transactions['AgeGroup'] = pd.cut(customer_transactions['Age'], bins=[0, 18, 35, 50, 65, 100], labels=['<18', '18-35', '35-50', '50-65', '65+'])

plt.figure(figsize=(12, 5))
sns.boxplot(x='AgeGroup', y='Transaction Amount', data=customer_transactions)
plt.title('Transaction Amount by Age Group')
plt.show()

In [ ]:
top_merchants = customer_transactions.groupby('Merchant Name')['Transaction Amount'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 5))
sns.barplot(x=top_merchants.values, y=top_merchants.index)
plt.title('Top 10 Merchants by Transaction Amount')
plt.show()

In [ ]:
category_spending = customer_transactions.groupby('Category')['Transaction Amount'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 5))
sns.barplot(x=category_spending.values, y=category_spending.index)
plt.title('Spending by Category')
plt.show()


In [ ]:
corr_matrix = customer_transactions[['Age', 'Transaction Amount']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

DATA PREPROCESSING

In [ ]:
customer_transactions['Gender'].unique()

In [ ]:
customer_transactions['Gender'] = customer_transactions['Gender'].replace ('F', 'Female')
customer_transactions['Gender'] = customer_transactions['Gender'].replace ('M', 'Male')
customer_transactions['Gender'] = customer_transactions['Gender'].replace ('nan', 'Prefer not to say')

In [ ]:
customer_transactions['Gender'].unique()

In [ ]:
customer_transactions['Transaction Amount'].unique()

In [ ]:
customer_transactions['Merchant Name'].unique()

UNIVARIATE ANALYSIS

In [ ]:
# Extract the values from the 'Gender' column and count occurrences
gender_counts = customer_transactions['Gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']

colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

# Create the pie chart
fig = px.pie(gender_counts, names='Gender', values='Count', title='Gender Distribution')

# Update the chart to add data labels
fig.update_traces(textinfo='percent+label')

# Show the plot
fig.show()

UNIVARIATE ANALYSIS

In [ ]:
# 1. Customer ID (int64)
print("Customer ID:")
print("Count of Unique Values:", customer_transactions['Customer ID'].nunique())
print("Summary Statistics:")
print(customer_transactions['Customer ID'].describe())

# 2. Name (object)
print("\nName:")
print("Count of Unique Names:", customer_transactions['Name'].nunique())
print("Most Common Names:")
print(customer_transactions['Name'].value_counts().head())
customer_transactions['Name_Length'] = customer_transactions['Name'].apply(len)
print("Mean Length:", customer_transactions['Name_Length'].mean())
print("Median Length:", customer_transactions['Name_Length'].median())
print("Missing Values:", customer_transactions['Name'].isnull().sum())

# 3. Surname (object)
print("\nSurname:")
print("Count of Unique Surnames:", customer_transactions['Surname'].nunique())
print("Most Common Surnames:")
print(customer_transactions['Surname'].value_counts().head())
customer_transactions['Surname_Length'] = customer_transactions['Surname'].apply(len)
print("Mean Length:", customer_transactions['Surname_Length'].mean())
print("Median Length:", customer_transactions['Surname_Length'].median())
print("Missing Values:", customer_transactions['Surname'].isnull().sum())

# 4. Gender (object)
print("\nGender:")
print("Count of Each Gender:")
print(customer_transactions['Gender'].value_counts())
print("Percentage Distribution:")
print(customer_transactions['Gender'].value_counts(normalize=True) * 100)
print("Missing Values:", customer_transactions['Gender'].isnull().sum())

# 5. Birthdate (object)
print("\nBirthdate:")
customer_transactions['Birth_Year'] = pd.to_datetime(customer_transactions['Birthdate']).dt.year
print("Distribution of Birth Years:")
print(customer_transactions['Birth_Year'].value_counts().sort_index())
print("Distribution of Birth Months:")
print(pd.to_datetime(customer_transactions['Birthdate']).dt.month.value_counts().sort_index())
customer_transactions['Age'] = pd.to_datetime('now').year - pd.to_datetime(customer_transactions['Birthdate']).dt.year
print("Mean Age:", customer_transactions['Age'].mean())
print("Median Age:", customer_transactions['Age'].median())
print("Youngest:", customer_transactions['Age'].min())
print("Oldest:", customer_transactions['Age'].max())
print("Missing Values:", customer_transactions['Birthdate'].isnull().sum())

# 6. Transaction Amount (float64)
print("\nTransaction Amount:")
print("Summary Statistics:")
print(customer_transactions['Transaction Amount'].describe())
plt.hist(customer_transactions['Transaction Amount'], bins=20)
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Transaction Amounts')
plt.show()
# Outlier detection (can be adjusted as needed)
outliers = customer_transactions[(customer_transactions['Transaction Amount'] - customer_transactions['Transaction Amount'].mean()).abs() > 3 * customer_transactions['Transaction Amount'].std()]
print("Number of Outliers:", len(outliers))

# 7. Date (object)
print("\nDate:")
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])
print("Distribution of Transactions Over Time:")
print(customer_transactions['Date'].dt.to_period('M').value_counts().sort_index())
print("Missing Values:", customer_transactions['Date'].isnull().sum())

# 8. Merchant Name (object)
print("\nMerchant Name:")
print("Count of Unique Merchant Names:", customer_transactions['Merchant Name'].nunique())
print("Most Common Merchant Names:")
print(customer_transactions['Merchant Name'].value_counts().head())
print("Missing Values:", customer_transactions['Merchant Name'].isnull().sum())

# 9. Category (object)
print("\nCategory:")
print("Count of Unique Categories:", customer_transactions['Category'].nunique())
print("Most Common Categories:")
print(customer_transactions['Category'].value_counts().head())
print("Missing Values:", customer_transactions['Category'].isnull().sum())

In [ ]:
# 4. Gender
print("\nGender:")
print("- Count of Each Gender:")
print(customer_transactions['Gender'].value_counts())
print("- Percentage Distribution:")
print(customer_transactions['Gender'].value_counts(normalize=True) * 100)
print("- Missing Values:", customer_transactions['Gender'].isnull().sum())

In [ ]:
# Columns to exclude from the loop
#exclude_columns = ['latitude', 'longitude', 'Pin code']

# Loop through each column
for column in customer_transactions.columns:
    # Skip excluded columns
   # if column in exclude_columns:
   #     continue
    
    # Get unique values and their counts
    value_counts = customer_transactions[column].value_counts()
    
    # Create bar chart
    fig = px.bar(value_counts, x=value_counts.index, y=value_counts.values, 
                 labels={'x': column, 'y': 'Count'}, 
                 title=f"Value Counts for {column}")
    
    # Show plot
    fig.show()

#### TASK 1 - CHECK FOR DUPLICATED ROWS

In [ ]:
Total_dup = customer_transactions.duplicated().sum()
print('The total number of duplicate rows in this dataset is', Total_dup)

#### TASK 2 - CHECK FOR MISSING VALUES

In [ ]:
customer_transactions.isna().sum()

In [ ]:
Total_MV = customer_transactions.isna().sum().sum()
print('The total number of missing values in this dataset is', Total_MV)

In [ ]:
#Check for the percentage for the missing values.

missing_gender = customer_transactions['Gender'].isnull().sum()
total_rows = len(customer_transactions)
missing_percentage = (missing_gender / total_rows) * 100

print(f"Missing Gender Values: {missing_gender} ({missing_percentage:.2f}%)")

#### HEATMAP FOR MISSING VALUES

In [ ]:
sns.heatmap(customer_transactions.isnull())

#### INSIGHTS

The results showed that there are 5,047 missing values in the gender column which sums up to 10.09% of the dataset. 

In dealing with this, I will replace all the missing values in the gender column to UNDEFINED.

This presupposes that the these customers did not find their gender orientation listed in there to select, hence the empty field.

#### DEALING WITH THE MISSING VALUES.

In [ ]:
#Replace missing values in the gender column with undefined.

customer_transactions['Gender'].fillna('Undefined', inplace=True)

In [ ]:
#Replace gender initials with the full word.

customer_transactions['Gender'] = customer_transactions['Gender'].replace('M', 'Male')
customer_transactions['Gender'] = customer_transactions['Gender'].replace('F', 'Female')

In [ ]:
#Checking for the unique values in the Gender column after replacing the missing values and renaming the initials with full word.
customer_transactions['Gender'].unique()

#### RENAME SOME COLUMNS NAME

In [ ]:
customer_transactions = customer_transactions.rename(columns={'Name': 'First Name', 'Surname': 'Last Name', 
                                                              'Birthdate': 'Date of Birth', 
                                                              'Transaction Amount': 'Transaction Amt ($)'})

In [ ]:
customer_transactions.columns

#### CHECK THE 

In [ ]:
customer_transactions.describe().T

In [ ]:
customer_transactions.describe(include='object').T

## EXPLORATORY DATA ANALYSIS, (E.D.A)

#### MAKE A COPY OF THE ORIGINAL DATASET.

In [ ]:
customer_transactions_copy = customer_transactions.copy

In [ ]:
Cols_drop = ['Customer ID', 'Last Name']
customer_transactions = customer_transactions.drop(columns= Cols_drop)

In [ ]:
customer_transactions.boxplot()

#### INSIGHTS TO BE DERIVED FROM THIS ANALYSIS

#### QUESTIONS

 - Which category made the highest sales during the period?
 - Which age grouping patronized more of cosmetics, clothings and travel?
 - Which date had the highest patronage for Clothing?

#### Question 1
 - Which category made the highest sales during the period?

In [ ]:
# Group the data by category and calculate total sales for each category
category_sales = customer_transactions.groupby('Category')['Transaction Amt ($)'].sum()

# Print the total sales for each category
print(category_sales)

In [ ]:
customer_transactions = category_sales.reset_index()
customer_transactions.columns = ['Category', 'Transaction Amt ($)']

# Plotting the bar chart using Plotly Express
fig = px.bar(customer_transactions, x='Category', y='Transaction Amt ($)',
             title='Total Sales by Category',
             labels={'Transaction Amt ($)': 'Transaction Amt ($)'},
             text='Transaction Amt ($)',
             width=800, height=500)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=45)

# Show the plot
fig.show()

In [ ]:
# Find the category with the highest total sales
highest_sales_category = category_sales.idxmax()
highest_sales_amount = category_sales.max()

print(f"The category with the highest sales is '{highest_sales_category}' with a total sales amount of $ {highest_sales_amount:.2f}")

#### Question 2
 - Which age group patronized more of cosmetics, clothings and travel?

In [ ]:
# Convert the 'Date of Birth' column to datetime
customer_transactions['Date of Birth'] = pd.to_datetime(customer_transactions['Date of Birth'])

# Calculate age based on the current date
current_date = datetime.now()
customer_transactions['Age'] = (current_date - customer_transactions['Date of Birth']) // pd.Timedelta(days=365)

customer_transactions

In [ ]:
# Group Ages into categories: young, middle age, and old

# Define age bins and labels
age_bins = [0, 30, 50, 100]  # You can adjust these age boundaries as needed

age_labels = ['Young', 'Middle Age', 'Old']

# Apply age grouping
customer_transactions['Age Group'] = pd.cut(customer_transactions['Age'], bins=age_bins, labels=age_labels, right=False)


In [ ]:
customer_transactions.head()

In [ ]:
# Create a pie chart to visualize the results
labels = ['Cosmetic', 'Clothing', 'Travel']
sizes = [grouped_data['Travel'].max(), grouped_data['Clothing'].max(), grouped_data['Cosmetic'].max()]
explode = (0.1, 0, 0)  # Explode the 1st slice (Cosmetic)

plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Age Groups Patronizing Products')

plt.show()

#### Question 3
 - Which date had the highest patronage for Clothing?

In [ ]:
# List of categories you want to analyze
categories_to_analyze = ['Cosmetic', 'Clothing', 'Travel' , 'Electronics' , 'Market', 'Restaurant']

# Iterate through the categories
for category in categories_to_analyze:
    # Filter the data for the specific category
    category_data = customer_transactions[customer_transactions['Category'] == category]
    
    # Group the data by date and count the number of purchases for each date
    date_patronage = category_data.groupby('Date').size()
    
    # Find the date with the highest patronage for the category
    date_with_highest_patronage = date_patronage.idxmax()
    highest_patronage_count = date_patronage.max()
    
    print(f"For the '{category}' category:")
    print(f"The date with the highest patronage is {date_with_highest_patronage} with {highest_patronage_count} purchases.")
    print()


 #### Question 4
 - What is the total number of person based on their gender patronized restaurant?

In [ ]:
# Filter the data for the "Restaurant" category
restaurant_data = customer_transactions[customer_transactions['Category'] == 'Restaurant']

# Group the data by gender and count the number of patrons for each gender
gender_patrons = restaurant_data['Gender'].value_counts()

print("Total number of people who patronized the restaurant based on gender:")
print(gender_patrons)


In [ ]:
customer_transactions = pd.DataFrame({'Gender': gender_patrons.index, 'Number of Patrons': gender_patrons.values})

# Creating a bar chart using Plotly Express
fig = px.bar(customer_transactions, x='Gender', y='Number of Patrons',
             color='Gender',
             title='Total Number of Restaurant Patrons by Gender',
             labels={'Number of Patrons': 'Number of Patrons', 'Gender': 'Gender'},
             width=1000, height=00)

# Show the plot
fig.show()

 #### Question 5
 - What is the trend in the transactions?

In [ ]:
# Convert the 'transaction_date' column to a datetime object
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Group the data by transaction date and count the number of transactions for each date
transaction_trend = customer_transactions.groupby('Date').size()

# Create a line chart to visualize the transaction trend
plt.figure(figsize=(12, 6))
plt.plot(transaction_trend.index, transaction_trend.values, marker='o', color='red')
plt.title('Transaction Trend Over The Transaction Period.')
plt.xlabel('Transaction Date')
plt.ylabel('Number of Transactions')
plt.grid(True)

# Optionally, format the x-axis labels for date
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.DayLocator(interval=10))  # Adjust the interval as needed

plt.xticks(rotation=45)

plt.show()

 #### Question 6
 - What is the percentage for the various gender? 

In [ ]:
#value counts for each gender

customer_transactions['Gender'].value_counts()

In [ ]:
# Assuming your DataFrame has a 'Gender' column, you can count the number of transactions for each gender
gender_counts = customer_transactions['Gender'].value_counts()

# Create a pie chart
plt.figure(figsize=(7, 7))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Gender Distribution Percentage')

# Add a legend
plt.legend()

plt.show()

In [ ]:
# Group by "Age Group" and count the number of each gender within each age group
age_gender_distribution = customer_transactions.groupby(['Age Group', 'Gender']).size().unstack(fill_value=0)

# Display the age-gender distribution
print(age_gender_distribution)

In [ ]:
# Plot the age-gender distribution as a bar chart
age_gender_distribution.plot(kind='bar')
plt.title('Age-Gender Distribution')
plt.xlabel('Age Group')
plt.ylabel('Count')

plt.show()

 #### Question 7
 - Which merchant name had the highest number of transactions during the period?

In [ ]:
# Group the data by the merchant and count the number of transactions for each merchant

merchant_transaction_counts = customer_transactions['Merchant Name'].value_counts()

merchant_with_highest_transactions = merchant_transaction_counts.idxmax()

highest_transaction_count = merchant_transaction_counts.max()

print(f"The merchant with the highest number of transactions is '{merchant_with_highest_transactions}'")
print()
print(f"They have {highest_transaction_count} transactions.")